## Part 2A: Optimisation and Machine Learning in Finance – Software - Solution

## Introduction

    In this linear programming method we are here to determine the portfolio of stock, bonds and options that maximises expected profit. We define the object functions as B, S and C. Then we create a linear programming formulation in the matrix format and substitute the constraints accordingly. Then a sequence of min and max pairs for each element in x, defining the minimum and maximum values of that decision variable is given. Since there is no bound we use None. All the decision variables are non negative. Once all the variales are assigned accordingly linprog function is used to find the optimal solution and potential losses. 

In [36]:
# Importing necessary libraries
import numpy as np
from scipy.optimize import linprog

# Expected profits per unit of investment
B=-10; S=-4; C=0;
c = np.array([B, S, C])

# Constraints in the left hand side matrix 
A = np.array([[90, 20, 1000],[0, 0, 1],[0, 0, -1],[-1, 0, 0],[0, -1, 0] ])

# Constraints in the following equation
b = np.array([20000, -50, 50, 0, 0])

# Bounds for variables
bounds = [(0, None), (0, None), (-50, 50)]

# Solve using the linprog to get the optimal solution
res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, 
              method='highs', 
              allback=None, 
              options=None, 
              x0=None, integrality=None)
print(res) #Result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -14000.0
              x: [ 0.000e+00  3.500e+03 -5.000e+01]
            nit: 0
          lower:  residual: [ 0.000e+00  3.500e+03  0.000e+00]
                 marginals: [ 8.000e+00  0.000e+00  2.000e+02]
          upper:  residual: [       inf        inf  1.000e+02]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              3.500e+03]
                 marginals: [-2.000e-01 -0.000e+00 -0.000e+00 -0.000e+00
                             -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0


## Result

    On solving the equations using the linprog solver we are getting an expected profit of £14,000. On analysing the result the optimal solutions for each of the decision variables B = 0, S = 3500 and C = -50. In this as a portfolio investor the profit is not always in positive. If the stock price increases we are bound to incur loss.

###########################################################################################################################

## Introduction

         In this Optimisation problem the investor wants to make a minimum profit at least 2000 pounds. That is why we are using gurobi pi to solve linear programming problem. Certain scenarios were already defined to us where investor is investing £20000 to invest in stocks, call option to buy 100 shares and a certain way to make stock investment a maximum profit from the three scenarios. We first started up by importing a gurobipy solver as gp and then we created a new model having number of attributes that can be queried or modified. This model is assigned to variable named as model_. This is then used to call the decision variables using addVar() function. The variables Bond, Stock and Call options represents investment and Profit1, Profit2 and Profit3 represents profits in various scenarios. The purpose of creating this is to add a variable to the model. It contains arguments inside a variable which we can define such as Lower bound value(lb) , Upper bound value (ub) , Objective coefficient(obj) , variable type, name and Initial coefficients for column.

In [1]:
# Importing necessary libraries
import gurobipy as gp

# Create a new gurobipy model
model_ = gp.Model(name='', env=None, cmodel=None)

# Creating variables for bond, stock, call and profit
Bond = model_.addVar(lb=0, name="Bond", ub=1e+100, obj=0.0, vtype='C',column=None) # Lower bound equal to 0 Bond investment
print(Bond)
Stock = model_.addVar(lb=0, name="Stock",ub=1e+100, obj=0.0, vtype='C',column=None) # Lower bound equal to 0 Investment of stock
print(Stock)
Call = model_.addVar(lb=-50, ub=50, name="Call", obj=0.0, vtype='C',column=None) # Lowerbound = -50 and Upperbound = 50 Investment in call option
print(Call)
Profit1 = model_.addVar(lb=0, name="Profit1", ub=1e+100, obj=0.0, vtype='C',column=None) # lower bound equal to 0 on profit scenario 1
print(Profit1)
Profit2 = model_.addVar(lb=0, name="Profit2", ub=1e+100, obj=0.0, vtype='C',column=None) # lower bound equal to 0 on profit scenario 2
print(Profit2)
Profit3 = model_.addVar(lb=0, name="Profit3", ub=1e+100, obj=0.0, vtype='C',column=None) # lower bound equal to 0 on profit scenario 3
print(Profit3)

Restricted license - for non-production use only - expires 2024-10-28
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>
<gurobi.Var *Awaiting Model Update*>


    The main idea of setting up the objective function after defining the variables are to maximize profits in various scenarios. Since we have taken 3 different scenarios that is why we divided the profits into a equal weightage of sums so that the profits can be maximized. The argument of this objective function contains linear expression of profits in 3 different scenarios taking into account the profits made and losses obtained in each of the scenarios. This routine will replace the 'Obj' attribute on model variables with the corresponding values from the supplied expression. This method uses the modelSense model attribute to determine the sense. Here since we are looking to maximize the profit we select GRB.MAXIMIZE sense.

In [2]:
# Set the model objective function
model_.setObjective((1/3)*Profit1 + (1/3)*Profit2 + (1/3)*Profit3, sense=gp.GRB.MAXIMIZE) # profit in each scenario, weighted equally.

    We then add the constraint function in such a way that the profit is maximized. In the first constraint we write a an equation which states that the amount of investment done while purchasing bonds, stock and call options which in total do not exceed the maximum investment. The second line of constraints are the investments done to express the profit in the scenario 1. The third line of constraints represents the investments done to express the profit in the scenario 2. The fourth line of constraints represents the investments done in third different way to express the profit in the scenario 3. The remaining three constarints are written in such a way that the profit is at least £2,000 in any of the three scenarios. In total there are seven different constraints which included budget, profit and riskless profit and the main purpose was to add constraints to the model.

In [11]:
# Adding constraints to the model
model_.addConstr(90*Bond + 20*Stock + 1000*Call <= 20000)  # LHS = Investment in RHS = Riskless profit
model_.addConstr(10*Bond + 20*Stock + 1500*Call == Profit1) # LHS = Investments made in RHS = Profit in scenario 1
model_.addConstr(10*Bond - 500*Call == Profit2) # LHS = Investments made in RHS = Profit in scenario 2
model_.addConstr(10*Bond - 8*Stock - 1000*Call == Profit3) # LHS = Investments made in RHS = Profit in scenario 3
model_.addConstr(Profit1 >= 2000)   # Least minimum profit
model_.addConstr(Profit2 >= 2000)   # Least minimum profit
model_.addConstr(Profit3 >= 2000)   # Least minimum profit

<gurobi.Constr *Awaiting Model Update*>

    Once the variable, model objective function and constraints are set up the process of optimization begins. The main goal in optimization is to find the variables where we can expect the maximumm profit. The optimization values tells how much investment we can make and what is the profit we can expect in each scenario.

In [4]:
# Optimization model
model_.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 6 columns and 17 nonzeros
Model fingerprint: 0x8788d020
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e-01, 3e-01]
  Bounds range     [5e+01, 5e+01]
  RHS range        [2e+03, 2e+04]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 4 rows, 6 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.6177223e+04   3.601495e+03   0.000000e+00      0s
       4    1.1200000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.120000000e+04


In [10]:
# Result of optimization 
print(f'The expected worth in 6 months is {model_.objVal + 20000}') # Expected profit
print(f'The total number of bonds to invest is {Bond.x}')  # Investment in bonds
print(f'The total number of stocks to invest is {Stock.x}') # Investment in stocks
print(f'The total number of stocks to cell {Call.x}')      # Investment in call options
print("Profit in scenario 1: ", Profit1.x)        # Scenario 1 profit
print("Profit in scenario 2: ", Profit2.x)        # Scenario 2 profit
print("Profit in scenario 3: ", Profit3.x)        # Scenario 3 profit

print("Expected Profit: ", 31200-20000)

The expected worth in 6 months is 31200.0
The total number of bonds to invest is 0.0
The total number of stocks to invest is 2800.0
The total number of stocks to cell -36.0
Profit in scenario 1:  2000.0
Profit in scenario 2:  18000.0
Profit in scenario 3:  13600.0
Expected Profit:  11200


## Result

    Finally after completion of optimization process, the values in decision variables tells us how much is the expected profit, how much can we invest in bonds, stocks and call options to get the maximum profit. Once the optimization process    is done, the result of the expected profit is given as an output. The output suggest that the expected worth in six months  for an investor will be £31,200 by investing in financial markets like bonds, stocks and call options. The optimal        investment that needs to be done in stocks to gain the maximum benefit is 2800, sell 36 call options and do zero investments in bonds. The profits in each scenarios are £2000, £18000 and £13600. The expected profit an investor can make is     £11200.

### References

Optimization Methods in Finance \
January 2007 \
DOI:10.1017/CBO9780511753886 \
SourceRePEc \
Publisher: Cambridge University PressISBN: 9780521861700

https://www.researchgate.net/publication/227390397_Optimization_Methods_in_Finance